<a href="https://colab.research.google.com/github/NikolasGialitsis/LDA2vec/blob/master/lda2vec_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
#!unzip /content/lda2vec.zip
#!ls
!unzip /content/lda2vec.zip

## Setup and Preprocessing

Install Dependencies



In [0]:
#!git clone https://github.com/maxent-ai/lda2vec

!pip install spacy
!pip install jellyfish
!pip install -r /content/lda2vec/requirements.txt 
!pip install pylda2vec


In [0]:
!python -m spacy download en

Import Libraries

In [8]:
%cd /content/lda2vec/
import os
import os.path
import pickle
import time
import shelve

import chainer
from chainer import cuda
from chainer import serializers
import chainer.optimizers as O
import numpy as np

from lda2vec import utils
from lda2vec import prepare_topics, print_top_words_per_topic, topic_coherence
from lda2vec import LDA2Vec
from lda2vec import preprocess, Corpus
#changed Preprocess.py line 'nlp = spacy.load("en_core_web_sm")' to solve error en not found
#and removed the import ... as en line 

import logging
logging.basicConfig()
import pickle
from sklearn.datasets import fetch_20newsgroups
import numpy as np
from lda2vec import preprocess, Corpus

/content/lda2vec


**CUDA** usage\
 *change Notebook Settings 'Hardware Acceleration' to 'GPU' to run*

In [9]:
gpu_id = int(os.getenv('CUDA_GPU', 0))
cuda.get_device(gpu_id).use()
print("Using GPU:" + str(gpu_id))

Using GPU:0


**Fetch Data**\
Set Stopwords
>>**DONE**: replace texts with covid19 collection

In [0]:
import json
with open('/content/db_entries.json') as f:
  data = json.loads(f.read())
titles = []
abstracts = []
concats = []
for entry in data:
  concat = (entry['joint_text']).split('------------------------------')  
  title = concat[0]
  abstract = concat[1]
  #print('title:',title)
  #print('abstract',abstract[:100])
  
  concats.append(concat[0] + ' ' + concat[1])
  titles.append(title)
  abstracts.append(abstract)

Pre-processing

*   Lowercase
*   Remove stopwords from nltk
*   Remove punctuation



In [191]:
import nltk 
import string
nltk.download('stopwords')
nltk.download('punkt')
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize 

stop_words = set(stopwords.words('english')) 
texts = []
for text in concats.copy():
  word_tokens = word_tokenize(text) 
  #remove stopwords
  text = ' '.join([w.lower() for w in word_tokens if not w.lower() in stop_words and len(w) >= 3]) 
  #remove punctuation
  #bioclean    = lambda t: re.sub('[.,?;*!%^&_+():-\[\]{}]', '', t.replace('"', '').replace('/', '').replace('\\', '').replace("'", '').strip().lower()).split()

  text = ''.join([w for w in text if w not in string.punctuation])
  print(text[:100])
  texts.append(text)
    


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
2019 novel coronavirus infection gastrointestinal tract since end december 2019 cluster patients pne
several suggestion operation colorectal cancer outbreak corona virus disease china pneumonia caused 
potential presymptomatic transmission sarscov2 zhejiang province china 2020 report 2family cluster p
straining system novel coronavirus covid19 preparedness concomitant disasters weeks first confirmed 
risk transportation 2019 novel coronavirus disease wuhan cities china january 2020 china quarantined
advances research cytokine storm mechanism induced corona virus disease 2019 corresponding immunothe
case 2019 novel coronavirus pregnant woman preterm delivery presented case 30week pregnant woman cov
diagnostic therapeutic strategies lung cancer patients outbreak

Preprocessing


*   set max word num per document
*   remove stopwords (aka. words in 'bad' list above)



In [0]:
# Preprocess data
max_length = 10000   # Limit of 10k words per document

Calculate tokens (list) and vocabulary (dictionary)\
OR\
Load them if already calculated

In [193]:
'''if a model is already trained,
skip to the cell loading vocab,corpus,flattened and doc_ids'''
tokens, vocab = preprocess.tokenize(texts, max_length, merge=False,n_threads=4)
'''LOAD IF ALREADY COMPUTED'''
#tokens = np.load("tokens.npy")
#vocab = np.load("vocab.npy")

/content/lda2vec/lda2vec/preprocess.py:77: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for row, doc in progress(enumerate(nlp.pipe(texts, **kwargs))):


/usr/lib/python3.6/runpy.py:193: DeprecationWarning: [W016] The keyword argument `n_threads` is now deprecated. As of v2.2.2, the argument `n_process` controls parallel inference via multiprocessing.
  "__main__", mod_spec)


'LOAD IF ALREADY COMPUTED'

run next line only if loaded

In [0]:
#vocab = vocab.tolist()

Bag-of-Words and Normalization
* remove really rare words
* downsample frequent words
* remove word gaps for words not in the vocabulary 
* flatten

In [0]:
corpus = Corpus()
# Make a ranked list of rare vs frequent words
corpus.update_word_count(tokens)
corpus.finalize()
# The tokenization uses spaCy indices, and so may have gaps
# between indices for words that aren't present in our dataset.
# This builds a new compact index
compact = corpus.to_compact(tokens)
# Remove extremely rare words
pruned = corpus.filter_count(compact, min_count=4)
# Convert the compactified arrays into bag of words arrays
bow = corpus.compact_to_bow(pruned)
# Words tend to have power law frequency, so selectively
# downsample the most prevalent words
clean = corpus.subsample_frequent(pruned)
# Now flatten a 2D array of document per row and word position
# per column to a 1D array of words. This will also remove skips
# and OoV words
doc_ids = np.arange(pruned.shape[0])
flattened, (doc_ids,) = corpus.compact_to_flat(pruned, doc_ids)

Pre-trained Word Vectors\
**TODO**: add pre-trained words when given

In [0]:
from gensim.models.word2vec import Word2Vec
from gensim.models import KeyedVectors
#!pip install gensim
assert flattened.min() >= 0
# Fill in the pretrained word vectors
n_dim = 30

import numpy as np
pretrained_filename= '/content/lda2vec/covid_embeddings/covid_19_w2v_embeds_30.model'


#vectors, s, f = corpus.compact_word_vectors(vocab, pretrained_filename,top=10027)
#model = Word2Vec.load(pretrained_filename)
#print(model.wv.syn0)

#print(vectors)

Save all files (uncomment for vectors if pre-trained included)

In [0]:
# Save all of the preprocessed files
pickle.dump(vocab, open('vocab.pkl', 'wb'))
pickle.dump(corpus, open('corpus.pkl', 'wb'))
np.save("flattened", flattened)
np.save("doc_ids", doc_ids)
np.save("pruned", pruned)
np.save("bow", bow)


In [0]:
np.save("vectors", vectors)

In [0]:
print(tokens.shape)
print(vocab)

(744, 10000)
{0: '2019', 1: 'novel', 2: 'coronavirus', 3: 'infection', 4: 'gastrointestinal', 5: 'tract', 6: 'since', 7: 'end', 8: 'december', 9: 'cluster', 10: 'patients', 11: 'pneumonia', 12: 'unknown', 13: 'origin', 14: 'reported', 15: 'wuhan', 16: 'hubei', 17: 'province', 18: 'china', 19: 'feb', 20: '17th', 21: '2020', 22: 'statistical', 23: 'data', 24: 'show', 25: 'outbreak', 26: 'constitutes', 27: 'epidemic', 28: 'threat', 29: 'exponential', 30: 'increase', 31: 'reached', 32: '75114', 33: 'confirmed', 34: 'cases', 35: '2239', 36: 'deaths', 37: 'different', 38: 'sarscov', 39: 'severe', 40: 'acute', 41: 'respiratory', 42: 'syndrome', 43: 'merscov', 44: 'middle', 45: 'east', 46: 'initial', 47: 'presentations', 48: 'chief', 49: 'complain', 50: 'covid19', 51: 'symptoms', 52: 'call', 53: 'upon', 54: 'firstline', 55: 'medical', 56: 'staff', 57: 'cautious', 58: 'pay', 59: 'attention', 60: 'untypical', 61: 'especially', 62: 'area', 63: 'besides', 64: 'viral', 65: 'nucleic', 66: 'acids', 6

In [197]:
%cd /content/lda2vec
#data_dir = os.getenv('data_dir', '../data/')
fn_vocab = 'vocab.pkl'
fn_corpus = 'corpus.pkl'
fn_flatnd = 'flattened.npy'
fn_docids = 'doc_ids.npy'
fn_vectors = 'vectors.npy'
vocab = pickle.load(open(fn_vocab, 'rb'))
corpus = pickle.load(open(fn_corpus, 'rb'))
flattened = np.load(fn_flatnd)
doc_ids = np.load(fn_docids)
vectors = np.load('/content/vectors.npy')

/content/lda2vec


##Defining Model Parameters



>> Parameters 
1.   Number of documents
2.   Number of unique words
3.   Dirichlet Prior 
4.   Number of topics
5.   Batchsize
6.   power for negative sampling 
7.   flag to include pre-trained or not
8.   sampling temperature
9.   number of dimensions of word vectors
10.  number of tokens in each document
11.  token frequencies



In [0]:
# Model Parameters
# Number of documents
n_docs = doc_ids.max() + 1
# Number of unique words in the vocabulary
n_vocab = flattened.max() + 1
# 'Strength' of the dircihlet prior; 200.0 seems to work well
clambda = 200.0
# Number of topics to fit
n_topics = int(os.getenv('n_topics', 10))
batchsize = 32
# Power for neg sampling
power = float(os.getenv('power', 0.75))
# Intialize with pretrained word vectors
pretrained = bool(int(os.getenv('pretrained', True)))
# Sampling temperature
temperature = float(os.getenv('temperature', 1.0))
# Number of dimensions in a single word vector
n_units = int(os.getenv('n_units', 30))
# Get the string representation for every compact key
words = corpus.word_list(vocab)[:n_vocab]
# How many tokens are in each document
doc_idx, lengths = np.unique(doc_ids, return_counts=True)
doc_lengths = np.zeros(doc_ids.max() + 1, dtype='int32')
doc_lengths[doc_idx] = lengths
# Count all token frequencies
tok_idx, freq = np.unique(flattened, return_counts=True)
term_frequency = np.zeros(n_vocab, dtype='int32')
term_frequency[tok_idx] = freq

In [0]:
for key in sorted(locals().keys()):
    val = locals()[key]
    if len(str(val)) < 100 and '<' not in str(val):
        print(key, val)

# Training the LDA2vec Model

In [0]:
model = LDA2Vec(n_documents=n_docs, n_document_topics=n_topics,
                n_units=n_units, n_vocab=n_vocab, counts=term_frequency,
                n_samples=15, power=power, temperature=temperature)

Add pretrained data

In [0]:

pretrained = True

'''
if os.path.exists('/content/lda2vec.hdf5'):
    print("Reloading from saved")
    serializers.load_hdf5("/content/lda2vec.hdf5", model)
#print(model.sampler.W.data)
'''
if pretrained:     
    model.sampler.W.data[:, :] = vectors[:n_vocab, :]

Adam Optimizer

In [213]:
model.to_gpu()
optimizer = O.Adam()
optimizer.setup(model)
clip = chainer.optimizer.GradientClipping(5.0)
optimizer.add_hook(clip)

/usr/local/lib/python3.6/dist-packages/chainer/optimizer.py:983: DeprecationWarning: chainer.optimizer.GradientClipping is deprecated from v4. Use chainer.optimizer_hooks.GradientClipping instead.
  DeprecationWarning)


In [0]:
j = 0
epoch = 0
fraction = batchsize * 1.0 / flattened.shape[0]
progress = shelve.open('progress.shelve')

Training with cuda


*   Measure coherence between topics every 100 iterations
*   this is for only one epoch
*   save the model
*   backpropagation style 



In [215]:
n_epochs = 1
for epoch in range(n_epochs):
    data = prepare_topics(cuda.to_cpu(model.mixture.weights.W.data).copy(),
                          cuda.to_cpu(model.mixture.factors.W.data).copy(),
                          cuda.to_cpu(model.sampler.W.data).copy(),
                          words)
    top_words = print_top_words_per_topic(data)
    if j % 100 == 0 and j > 100:
        coherence = topic_coherence(top_words)
        for j in range(n_topics):
            print(j, coherence[(j, 'cv')])
        kw = dict(top_words=top_words, coherence=coherence, epoch=epoch)
        progress[str(epoch)] = pickle.dumps(kw)
    data['doc_lengths'] = doc_lengths
    data['term_frequency'] = term_frequency
    np.savez('topics.pyldavis', **data)
    print(epoch)
    for d, f in utils.chunks(batchsize, doc_ids, flattened):
        t0 = time.time()
        model.cleargrads()
        #optimizer.use_cleargrads(use=False)
        l = model.fit_partial(d.copy(), f.copy())
        print("after partial fitting:", l)
        prior = model.prior()
        loss = prior * fraction
        loss.backward()
        optimizer.update()
        msg = ("J:{j:05d} E:{epoch:05d} L:{loss:1.3e} "
               "P:{prior:1.3e} R:{rate:1.3e}")
        prior.to_cpu()
        loss.to_cpu()
        t1 = time.time()
        dt = t1 - t0
        rate = batchsize / dt
        logs = dict(loss=float(l), epoch=epoch, j=j,
                    prior=float(prior.data), rate=rate)
        print(msg.format(**logs))
        j += 1
    serializers.save_hdf5("lda2vec.hdf5", model)

Top words in topic 0 hong identity united google search per million similarity phylogenetic homology
Top words in topic 1 epithelial morphology tropism wall reveal alveolar thickening epithelia play intestinal
Top words in topic 2 diagnose identify tool understand detect contribute treat understanding predict improve
Top words in topic 3 kinase cases deaths sequences version syndrome genes transcription cancers domain
Top words in topic 4 staining activation release inhibition infiltration formation activity microscopy damage cells
Top words in topic 5 january extracted february march collected december jan summarizes acids april
Top words in topic 6 role entry pathogenesis protease asthma infection influenza potent therapy antivirals
Top words in topic 7 able used needed considered aware important implemented essential effective required
Top words in topic 8 vero detected viral virus coronaviruses coronavirus rna tropism infect characterization
Top words in topic 9 attitudes variables

In [0]:
#!zip -r myzip.zip /content/lda2vec
#from google.colab import files
#files.download('myzip.zip')

In [216]:
npz = np.load(open('/content/lda2vec/topics.pyldavis.npz', 'rb'))
dat = {k: v for (k, v) in npz.items()}
dat['vocab'] = dat['vocab'].tolist()
print(dat['vocab'])

['out_of_vocabulary', 'out_of_vocabulary', '<SKIP>', 'covid19', 'patients', 'coronavirus', 'disease', 'cases', '2019', 'china', 'sarscov2', 'clinical', 'health', 'respiratory', 'infection', 'outbreak', 'severe', 'novel', '2020', 'wuhan', 'epidemic', 'treatment', 'pneumonia', 'virus', 'acute', 'results', 'transmission', 'syndrome', 'control', 'confirmed', 'data', 'february', 'spread', 'risk', 'days', 'study', 'public', 'case', 'symptoms', 'methods', 'reported', 'care', 'infected', 'viral', 'number', 'management', 'may', 'pandemic', 'first', 'measures', 'group', 'countries', 'also', 'prevention', 'early', 'january', 'characteristics', 'caused', 'diagnosis', 'patient', 'findings', 'medical', 'emergency', 'new', '2019ncov', 'world', 'hubei', 'among', 'including', 'time', 'enditag', 'itag', 'hospital', 'december', 'chest', 'two', 'based', 'analysis', 'high', 'higher', 'features', 'using', 'current', 'positive', 'lung', 'fever', 'province', 'potential', 'epidemiological', 'global', 'period',

Print top 10 words for each topic

In [217]:
top_n = 100
topic_to_topwords = {}
for j, topic_to_word in enumerate(dat['topic_term_dists']):
    top = np.argsort(topic_to_word)[::-1][:top_n]
    msg = 'Topic %i '  % j
    top_words = [dat['vocab'][i].strip()[:35] for i in top]
    msg += ' '.join(top_words)
    print(msg)
    topic_to_topwords[j] = top_words

Topic 0 hong identity united google search per million similarity phylogenetic homology version discovery sequences databases usage distance arabia 000 searches fatality 2020 enc genomes news covid server pubmed number classification diversity us confidence millions publicly sequence prise market sharing 1000 internet emergence value transmission european pandemia reference website evolutionary emerging estimates republic zoonoses sequencing 3cl threat transmissibility forecasting zoonotic accuracy repurposing set users international ebola rights 8000 uncertainty chinese families online embase modeling detection publication prediction modelling kcalmol new forecasts estimation travel cfr 2019 genome consensus north algorithm access bat january updated cost demand searched decision computational pandemic size docking framework
Topic 1 epithelial morphology tropism wall reveal alveolar thickening epithelia play intestinal electron microbiota properties bronchial airway phylogenetic struc

## Visualize Topics 
>> lower λ to get rare words appearing only on selected topics

In [218]:
!pip install pyLDAvis
import pyLDAvis
pyLDAvis.enable_notebook()
#warnings.filterwarnings('ignore')
import math

#dat['topic_term_dists'] = dat['topic_term_dists'] / dat['topic_term_dists'].sum(axis=1)[:, None]
prepared_data = pyLDAvis.prepare(dat['topic_term_dists'], dat['doc_topic_dists'], 
                                 dat['doc_lengths'] * 1.0, dat['vocab'], dat['term_frequency'] * 1.0, mds='tsne')

In [219]:
pyLDAvis.display(prepared_data)

## Infer topics for new document

Parse new doc

In [220]:
#corpus = Corpus()

import json
with open('/content/new_document.json') as f:
  data = json.loads(f.read())
titles = []
abstracts = []
concats = []
for entry in data:
  concat = (entry['joint_text']).split('------------------------------')  
  title = concat[0]
  abstract = concat[1]
  print('title:',title)
  print('abstract',abstract[:100])
  concats.append(concat[0] + ' ' + concat[1])
  titles.append(title)
  abstracts.append(abstract)



title: [The treatment proposal for the patients with breast diseases in the central epidemic area of 2019 coronavirus disease].

abstract 
Currently, the epidemic of 2019 coronavirus disease (COVID-19) is still ongoing. The characteristic


Preprocess new doc


In [0]:
stop_words = set(stopwords.words('english')) 
new_text = []
for text in concats.copy():
  word_tokens = word_tokenize(text) 
  #remove stopwords
  text = ' '.join([w.lower() for w in word_tokens if not w.lower() in stop_words and len(w) >= 3]) 
  #remove punctuation
  #bioclean    = lambda t: re.sub('[.,?;*!%^&_+():-\[\]{}]', '', t.replace('"', '').replace('/', '').replace('\\', '').replace("'", '').strip().lower()).split()

  text = ''.join([w for w in text if w not in string.punctuation])
  new_text.append(text) 


In [222]:
max_length = 10000
new_tokens, new_vocab = preprocess.tokenize(new_text, max_length, merge=False,n_threads=4)


/content/lda2vec/lda2vec/preprocess.py:77: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for row, doc in progress(enumerate(nlp.pipe(texts, **kwargs))):


/usr/lib/python3.6/runpy.py:193: DeprecationWarning: [W016] The keyword argument `n_threads` is now deprecated. As of v2.2.2, the argument `n_process` controls parallel inference via multiprocessing.
  "__main__", mod_spec)


In [223]:
if os.path.exists('/content/lda2vec.hdf5'):
    print("Reloading from saved")
    serializers.load_hdf5("/content/lda2vec.hdf5", model)

#model.sampler.W.data[:, :] = new_text[:n_vocab, :]
data = prepare_topics(cuda.to_cpu(model.mixture.weights.W.data).copy(),
                      cuda.to_cpu(model.mixture.factors.W.data).copy(),
                      cuda.to_cpu(model.sampler.W.data).copy(),
                      words)
#add new randomized vector for new doc

random_doc_topic = np.random.dirichlet(np.ones(10),size=1)
first_doc = data['doc_topic_dists'][0]

num_docs = data['doc_topic_dists'].shape[0]
data['doc_topic_dists'] = np.append(arr=data['doc_topic_dists'],values=random_doc_topic)
data['doc_topic_dists'] = data['doc_topic_dists'].reshape(num_docs+1,n_topics)

print(data['doc_topic_dists'].shape)
print('random_doc = ',random_doc_topic)


(745, 10)
random_doc =  [[0.30630258 0.04174011 0.27650185 0.01615159 0.02138856 0.04157715
  0.10140192 0.00304826 0.17539766 0.01649033]]


In [224]:
model.to_gpu()
optimizer = O.Adam()
optimizer.setup(model)
clip = chainer.optimizer.GradientClipping(5.0)
optimizer.add_hook(clip)
j = 0
epoch = 0
fraction = batchsize * 1.0 / flattened.shape[0]
progress = shelve.open('progress.shelve')

/usr/local/lib/python3.6/dist-packages/chainer/optimizer.py:983: DeprecationWarning: chainer.optimizer.GradientClipping is deprecated from v4. Use chainer.optimizer_hooks.GradientClipping instead.
  DeprecationWarning)


In [225]:
n_epochs = 1

for epoch in range(1):
    data = prepare_topics(cuda.to_cpu(model.mixture.weights.W.data).copy(),
                          cuda.to_cpu(model.mixture.factors.W.data).copy(),
                          cuda.to_cpu(model.sampler.W.data).copy(),
                          words)
    top_words = print_top_words_per_topic(data)
    data['doc_lengths'] = doc_lengths
    data['term_frequency'] = term_frequency
    #print(data)
    for d, f in utils.chunks(batchsize, np.array(doc_ids), flattened):
        t0 = time.time()
        model.cleargrads()
        #optimizer.use_cleargrads(use=False)
        l = model.fit_partial(d.copy(), f.copy(),update_only_docs=True)
        print(l)
        print("after partial fitting:", l)
        prior = model.prior()
        loss = prior * fraction
        loss.backward()
        optimizer.update()
        msg = ("J:{j:05d} E:{epoch:05d} L:{loss:1.3e} "
               "P:{prior:1.3e} R:{rate:1.3e}")
        prior.to_cpu()
        loss.to_cpu()
        t1 = time.time()
        dt = t1 - t0
        rate = batchsize / dt
        logs = dict(loss=float(l), epoch=epoch, j=j,
                    prior=float(prior.data), rate=rate)
        print(msg.format(**logs))
        j += 1

Streaming output truncated to the last 5000 lines.
J:00760 E:00000 L:1.936e+04 P:-1.897e+04 R:1.036e+03
26596.656
after partial fitting: 26596.656
J:00761 E:00000 L:2.660e+04 P:-1.897e+04 R:1.032e+03
21364.219
after partial fitting: 21364.219
J:00762 E:00000 L:2.136e+04 P:-1.897e+04 R:8.952e+02
19612.844
after partial fitting: 19612.844
J:00763 E:00000 L:1.961e+04 P:-1.897e+04 R:8.790e+02
17979.371
after partial fitting: 17979.371
J:00764 E:00000 L:1.798e+04 P:-1.898e+04 R:1.024e+03
28802.195
after partial fitting: 28802.195
J:00765 E:00000 L:2.880e+04 P:-1.898e+04 R:1.002e+03
20046.23
after partial fitting: 20046.23
J:00766 E:00000 L:2.005e+04 P:-1.898e+04 R:9.786e+02
16962.137
after partial fitting: 16962.137
J:00767 E:00000 L:1.696e+04 P:-1.898e+04 R:1.023e+03
19752.758
after partial fitting: 19752.758
J:00768 E:00000 L:1.975e+04 P:-1.898e+04 R:9.656e+02
13431.695
after partial fitting: 13431.695
J:00769 E:00000 L:1.343e+04 P:-1.899e+04 R:1.036e+03
20203.729
after partial fitting: 2

In [231]:
top_words = print_top_words_per_topic(data)

Top words in topic 0 inhibits showed exhibited des containing glycoprotein protein receptor staining sem
Top words in topic 1 inhibits showed exhibited des containing glycoprotein receptor protein staining plays
Top words in topic 2 inhibits showed exhibited des containing glycoprotein receptor protein plays induced
Top words in topic 3 inhibits showed exhibited des containing glycoprotein receptor protein staining plays
Top words in topic 4 inhibits showed exhibited des containing glycoprotein protein receptor staining induced
Top words in topic 5 inhibits showed exhibited des containing glycoprotein receptor protein staining induced
Top words in topic 6 inhibits showed exhibited des containing glycoprotein receptor protein staining induced
Top words in topic 7 inhibits showed exhibited des containing glycoprotein receptor protein cells staining
Top words in topic 8 inhibits showed exhibited des containing glycoprotein protein receptor staining induced
Top words in topic 9 inhibits sh

In [227]:
print('==== MAKE SURE THAT THE OTHER DOCS DONT CHANGE')
print(first_doc)
print(data['doc_topic_dists'][0])
print('\n==================================')
print('==== MAKE SURE THAT THE NEW DOC HAS CHANGED')
print(random_doc_topic)
print(data['doc_topic_dists'][-1])

==== MAKE SURE THAT THE OTHER DOCS DONT CHANGE
[0.09949633 0.09379722 0.1034795  0.09723914 0.10922438 0.10073396
 0.09992947 0.09778783 0.10782836 0.0904838 ]
[0.09949633 0.09379722 0.1034795  0.09723914 0.10922438 0.10073396
 0.09992947 0.09778783 0.10782836 0.0904838 ]

==== MAKE SURE THAT THE NEW DOC HAS CHANGED
[[0.30630258 0.04174011 0.27650185 0.01615159 0.02138856 0.04157715
  0.10140192 0.00304826 0.17539766 0.01649033]]
[0.09487368 0.09560376 0.10308416 0.09804277 0.12193209 0.10402615
 0.09163371 0.09199105 0.09543919 0.10337348]


In [228]:
t=0
print('New Document Composition\n')
for d in data['doc_topic_dists'][-1]:
  print('\t',round(d*100,2),'\t percent of Topic',t)
  t+=1 

New Document Composition

	 9.49 	 percent of Topic 0
	 9.56 	 percent of Topic 1
	 10.31 	 percent of Topic 2
	 9.8 	 percent of Topic 3
	 12.19 	 percent of Topic 4
	 10.4 	 percent of Topic 5
	 9.16 	 percent of Topic 6
	 9.2 	 percent of Topic 7
	 9.54 	 percent of Topic 8
	 10.34 	 percent of Topic 9
